# Diego Cerretti 2024280040 - Homework 2 EM Algorithm

We implement an EM algorithm for a mixture of multinomials model, aimed at discovering topics in the 20 Newsgroups dataset by clustering documents and extracting the most frequent words in each topic.

## Data Loading and Preprocessing

In [79]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from scipy.special import logsumexp

In [81]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
vectorizer = TfidfVectorizer(max_df=0.5, min_df=20, stop_words='english', max_features=10000)
X = vectorizer.fit_transform(newsgroups.data)
T = X.toarray().astype(float)
vocab = vectorizer.get_feature_names_out()
D, W = T.shape

## Parameter Initialization

In [89]:
def initialize_parameters(K, T):
    np.random.seed(42)
    kmeans = KMeans(n_clusters=K, n_init=10, random_state=42).fit(T)
    cluster_assignments = kmeans.labels_
    pi = np.bincount(cluster_assignments, minlength=K) / len(cluster_assignments)
    mu = np.zeros((K, W))
    for k in range(K):
        mu[k] = T[cluster_assignments == k].mean(axis=0)
    mu += 1e-15  
    mu /= mu.sum(axis=1, keepdims=True)
    return pi, mu

## Expectation Step

In [91]:
def e_step(T, pi, mu):
    log_pi = np.log(pi + 1e-15)
    log_mu = np.log(mu + 1e-15)
    log_gamma = log_pi[np.newaxis, :] + T @ log_mu.T
    log_gamma -= logsumexp(log_gamma, axis=1)[:, np.newaxis]
    gamma = np.exp(log_gamma)
    return gamma

## Maximization Step

In [92]:
def m_step(T, gamma):
    pi = gamma.sum(axis=0) / gamma.shape[0]
    mu = (gamma.T @ T) + 1e-15
    mu /= mu.sum(axis=1, keepdims=True)  # Normalize
    return pi, mu

## Log-Likelihoog Calculation

In [95]:
def compute_log_likelihood(T, pi, mu):
    log_pi = np.log(pi + 1e-15)
    log_mu = np.log(mu + 1e-15)
    log_likelihood = np.sum(logsumexp(log_pi + T @ log_mu.T, axis=1))
    return log_likelihood / T.shape[0]

## EM Algorithm Execution

In [97]:
def run_em(T, K, max_iters=50, tol=1e-4):
    pi, mu = initialize_parameters(K, T)
    log_likelihoods = []
    for iteration in range(max_iters):
        gamma = e_step(T, pi, mu)
        pi, mu = m_step(T, gamma)
        log_likelihood = compute_log_likelihood(T, pi, mu)
        log_likelihoods.append(log_likelihood)
        if iteration > 0 and abs(log_likelihoods[-1] - log_likelihoods[-2]) < tol:
            print(f'Converged at iteration {iteration}')
            break
        if iteration % 5 == 0:
            print(f'Iteration {iteration}, Avg Log-Likelihood: {log_likelihood}')
    return pi, mu

## Extracting Top Words

In [99]:
def get_top_words(mu, vocab, n_top_words=10):
    return [[vocab[i] for i in topic.argsort()[-n_top_words:][::-1]] for topic in mu]

## Running and Displaying Results

In [101]:
K_values = [10, 20, 30, 50]
results = {}

for K in K_values:
    print(f'\nRunning EM algorithm for K={K}')
    pi, mu = run_em(T, K, max_iters=50, tol=1e-4)
    results[K] = get_top_words(mu, vocab)

# Display results
for K in K_values:
    print(f'\nTop words for K={K}:')
    for idx, words in enumerate(results[K]):
        print(f"Topic {idx+1}: {', '.join(words)}")


Running EM algorithm for K=10
Iteration 0, Avg Log-Likelihood: -42.728953534971396
Iteration 5, Avg Log-Likelihood: -42.56777007690151
Iteration 10, Avg Log-Likelihood: -42.55073955654128
Iteration 15, Avg Log-Likelihood: -42.543371064036464
Iteration 20, Avg Log-Likelihood: -42.538125104037924
Iteration 25, Avg Log-Likelihood: -42.53415182619423
Iteration 30, Avg Log-Likelihood: -42.52871235214928
Iteration 35, Avg Log-Likelihood: -42.52008942834828
Iteration 40, Avg Log-Likelihood: -42.51826422822769
Iteration 45, Avg Log-Likelihood: -42.51696649725123

Running EM algorithm for K=20
Iteration 0, Avg Log-Likelihood: -42.568482524502215
Iteration 5, Avg Log-Likelihood: -42.306126347448235
Iteration 10, Avg Log-Likelihood: -42.255950171189305
Iteration 15, Avg Log-Likelihood: -42.23946379851101
Iteration 20, Avg Log-Likelihood: -42.23105982383621
Iteration 25, Avg Log-Likelihood: -42.22499778344842
Iteration 30, Avg Log-Likelihood: -42.220687261247576
Iteration 35, Avg Log-Likelihood: 